In [0]:
"""! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
import setup_google_colab
# please, uncomment the week you're working on
# setup_google_colab.setup_week1()  
# setup_google_colab.setup_week2()
# setup_google_colab.setup_week3()
# setup_google_colab.setup_week4()
# setup_google_colab.setup_project()
setup_google_colab.setup_honor()"""

"! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py\nimport setup_google_colab\n# please, uncomment the week you're working on\n# setup_google_colab.setup_week1()  \n# setup_google_colab.setup_week2()\n# setup_google_colab.setup_week3()\n# setup_google_colab.setup_week4()\n# setup_google_colab.setup_project()\nsetup_google_colab.setup_honor()"

In [0]:
%cd "/content/drive/My Drive/HonorProject/NLP Honor"

/content/drive/My Drive/HonorProject/NLP Honor


In [0]:
########## select max_len of 100 or 200 #####################################
########## from cornell, opensubs, scotus ############################

In [0]:
import os
import pickle
import pandas as pd
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.utils import to_categorical

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply, LayerNormalization
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Reshape, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model, Model, Sequential
import tensorflow.keras.backend as K

"""import keras
from keras.layers import Layer
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply #, LayerNormalization
from keras.layers import RepeatVector, Dense, Activation, Lambda, Reshape, Embedding, Dropout
from keras.optimizers import Adam
from keras.models import load_model, Model, Sequential
import keras.backend as K"""

import numpy as np
import pandas as pd
import random
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import nltk
nltk.download('punkt')

from tqdm import tqdm
#from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
!pip show tensorflow

Name: tensorflow
Version: 2.2.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, protobuf, h5py, wrapt, gast, google-pasta, tensorflow-estimator, scipy, absl-py, keras-preprocessing, opt-einsum, grpcio, wheel, astunparse, termcolor, tensorboard, numpy
Required-by: fancyimpute


In [0]:
from datasets import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
"""choices=["cornell", "scotus", "opensubs"] #, "ubuntu"]
Classes = [CornellData, ScotusData, OpensubsData] #, UbuntuData]

for choice, Class in zip(choices, Classes):
    print(choice, Class, "started")
    dataset_path = os.path.join("data", choice)
    dataset = Class(dataset_path)
    conversations = dataset.getConversations()

    with open("conversations/d_" + str(choice) + '_convs.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(conversations, f, pickle.HIGHEST_PROTOCOL)

    print(choice, Class, "dumped")
    print()
"""

'choices=["cornell", "scotus", "opensubs"] #, "ubuntu"]\nClasses = [CornellData, ScotusData, OpensubsData] #, UbuntuData]\n\nfor choice, Class in zip(choices, Classes):\n    print(choice, Class, "started")\n    dataset_path = os.path.join("data", choice)\n    dataset = Class(dataset_path)\n    conversations = dataset.getConversations()\n\n    with open("conversations/d_" + str(choice) + \'_convs.pickle\', \'wb\') as f:\n        # Pickle the \'data\' dictionary using the highest protocol available.\n        pickle.dump(conversations, f, pickle.HIGHEST_PROTOCOL)\n\n    print(choice, Class, "dumped")\n    print()\n'

In [0]:
"""choices=["cornell", "scotus", "opensubs"]#, "ubuntu"]

conversations={}
for choice in choices:
    print(choice, "size: ")

    with open("conversations/d_" + str(choice) + '_convs.pickle', 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        conversations[choice] = pickle.load(f)

    print(len(conversations[choice]))
"""

'choices=["cornell", "scotus", "opensubs"]#, "ubuntu"]\n\nconversations={}\nfor choice in choices:\n    print(choice, "size: ")\n\n    with open("conversations/d_" + str(choice) + \'_convs.pickle\', \'rb\') as f:\n        # The protocol version used is detected automatically, so we do not\n        # have to specify it.\n        conversations[choice] = pickle.load(f)\n\n    print(len(conversations[choice]))\n'

In [0]:
max_word_len=25

In [0]:
"""choices=["cornell", "scotus", "opensubs"]#, "ubuntu"]

datas = []
for choice in choices:
    data = splitConversations(conversations[choice], max_len, fast_preprocessing=True)
    datas.extend(data)

print("total convs:", len(datas))

with open("conversations/dataset"+str(max_len)+'chars_without_ubuntu.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)
"""

'choices=["cornell", "scotus", "opensubs"]#, "ubuntu"]\n\ndatas = []\nfor choice in choices:\n    data = splitConversations(conversations[choice], max_len, fast_preprocessing=True)\n    datas.extend(data)\n\nprint("total convs:", len(datas))\n\nwith open("conversations/dataset"+str(max_len)+\'chars_without_ubuntu.pickle\', \'wb\') as f:\n        # Pickle the \'data\' dictionary using the highest protocol available.\n        pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)\n'

In [0]:
"""with open("conversations/dataset"+str(max_len)+'chars_without_ubuntu.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    datas = pickle.load(f)

print(len(datas))"""

'with open("conversations/dataset"+str(max_len)+\'chars_without_ubuntu.pickle\', \'rb\') as f:\n    # The protocol version used is detected automatically, so we do not\n    # have to specify it.\n    datas = pickle.load(f)\n\nprint(len(datas))'

In [0]:
"""import pandas as pd
columns = ("request", "reply", "line1", "line2")
dataframe = pd.DataFrame(data=datas, columns=columns)

for i in range(5):
    dataframe = dataframe.sample(frac=1)
dataframe.to_csv(path_or_buf="conversations/datasetDF"+str(max_len)+'chars_without_ubuntu.txt', index=False)"""

'import pandas as pd\ncolumns = ("request", "reply", "line1", "line2")\ndataframe = pd.DataFrame(data=datas, columns=columns)\n\nfor i in range(5):\n    dataframe = dataframe.sample(frac=1)\ndataframe.to_csv(path_or_buf="conversations/datasetDF"+str(max_len)+\'chars_without_ubuntu.txt\', index=False)'

In [0]:
"""dataframe = pd.read_csv("conversations/datasetDF"+str(max_len)+'chars_without_ubuntu.txt')
print(dataframe)"""

'dataframe = pd.read_csv("conversations/datasetDF"+str(max_len)+\'chars_without_ubuntu.txt\')\nprint(dataframe)'

In [0]:
"""max_word_len = 25
choices=["cornell"]#, "scotus", "opensubs"]#, "ubuntu"]

conversations={}
for choice in choices:
    print(choice, "size: ")

    with open("conversations/d_" + str(choice) + '_convs.pickle', 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        conversations[choice] = pickle.load(f)

    print(len(conversations[choice]))

datas = []
for choice in choices:
    data = splitConversations(conversations[choice], max_word_len, fast_preprocessing=False)
    datas.extend(data)

print("total convs:", len(datas))


print(datas[0])

with open("conversations/dataset_nltk_"+str(max_len)+'words_only_cornell.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)

import pandas as pd
columns = ("request", "reply", "line1", "line2")
dataframe = pd.DataFrame(data=datas, columns=columns)

for i in range(5):
    dataframe = dataframe.sample(frac=1)
dataframe.to_csv(path_or_buf="conversations/dataset_nltk_DF"+str(max_len)+'words_only_cornell.txt', index=False)"""

'max_word_len = 25\nchoices=["cornell"]#, "scotus", "opensubs"]#, "ubuntu"]\n\nconversations={}\nfor choice in choices:\n    print(choice, "size: ")\n\n    with open("conversations/d_" + str(choice) + \'_convs.pickle\', \'rb\') as f:\n        # The protocol version used is detected automatically, so we do not\n        # have to specify it.\n        conversations[choice] = pickle.load(f)\n\n    print(len(conversations[choice]))\n\ndatas = []\nfor choice in choices:\n    data = splitConversations(conversations[choice], max_word_len, fast_preprocessing=False)\n    datas.extend(data)\n\nprint("total convs:", len(datas))\n\n\nprint(datas[0])\n\nwith open("conversations/dataset_nltk_"+str(max_len)+\'words_only_cornell.pickle\', \'wb\') as f:\n        # Pickle the \'data\' dictionary using the highest protocol available.\n        pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)\n\nimport pandas as pd\ncolumns = ("request", "reply", "line1", "line2")\ndataframe = pd.DataFrame(data=datas, columns

In [0]:
max_word_len = 25


In [0]:
max_word_len = 25

with open("conversations/dataset_nltk_"+str(max_word_len)+'words_only_cornell.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    datas = pickle.load(f)

print(len(datas))

169273


In [0]:
' '.join(datas[0][1])

"i do n't fucking know . what am i , his biographer ? olaf , what part of russia are you from ?"

In [0]:
"""tok_word = text.Tokenizer(lower=True, oov_token="<OOV>")
tok_word.fit_on_texts(list(np.array(datas)[:,0])+list(np.array(datas)[:,1]))
word_index = tok_word.word_index
print(len(word_index))
inv_word_index = {v:k for k,v in word_index.items()}

START_TOKEN, END_TOKEN = len(word_index)+1, len(word_index)+2

# Vocabulary size plus start and end token
VOCAB_SIZE = len(word_index) + 3"""

'tok_word = text.Tokenizer(lower=True, oov_token="<OOV>")\ntok_word.fit_on_texts(list(np.array(datas)[:,0])+list(np.array(datas)[:,1]))\nword_index = tok_word.word_index\nprint(len(word_index))\ninv_word_index = {v:k for k,v in word_index.items()}\n\nSTART_TOKEN, END_TOKEN = len(word_index)+1, len(word_index)+2\n\n# Vocabulary size plus start and end token\nVOCAB_SIZE = len(word_index) + 3'

In [0]:
"""tok_pickle = {}

tok_pickle["tok_word"] = tok_word
tok_pickle["word_index"] = word_index
tok_pickle["inv_word_index"] = inv_word_index
tok_pickle["START_TOKEN"] = START_TOKEN
tok_pickle["END_TOKEN"] = END_TOKEN
tok_pickle["VOCAB_SIZE"] = VOCAB_SIZE

with open("conversations/dataset_nltk_tokenizer_"+str(max_word_len)+'words_only_cornell.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(tok_pickle, f, pickle.HIGHEST_PROTOCOL)"""

'tok_pickle = {}\n\ntok_pickle["tok_word"] = tok_word\ntok_pickle["word_index"] = word_index\ntok_pickle["inv_word_index"] = inv_word_index\ntok_pickle["START_TOKEN"] = START_TOKEN\ntok_pickle["END_TOKEN"] = END_TOKEN\ntok_pickle["VOCAB_SIZE"] = VOCAB_SIZE\n\nwith open("conversations/dataset_nltk_tokenizer_"+str(max_word_len)+\'words_only_cornell.pickle\', \'wb\') as f:\n        # Pickle the \'data\' dictionary using the highest protocol available.\n        pickle.dump(tok_pickle, f, pickle.HIGHEST_PROTOCOL)'

In [0]:
with open("conversations/dataset_nltk_tokenizer_"+str(max_word_len)+'words_only_cornell.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    tok_pickle = pickle.load(f)

tok_word = tok_pickle["tok_word"]
word_index = tok_pickle["word_index"] 
inv_word_index = tok_pickle["inv_word_index"]
START_TOKEN = tok_pickle["START_TOKEN"]
END_TOKEN = tok_pickle["END_TOKEN"] 
VOCAB_SIZE = tok_pickle["VOCAB_SIZE"] 

In [0]:
DFrequests = np.array(datas)[:,0]
DFreplys = np.array(datas)[:,1]

del datas

In [0]:
max_seq_len = max_word_len+2


In [0]:
max_seq_len = max_word_len+2
DF_req_sequences = tok_word.texts_to_sequences(DFrequests)
del DFrequests

for i in range(len(DF_req_sequences)):
    DF_req_sequences[i] = [START_TOKEN] + DF_req_sequences[i] + [END_TOKEN]    

DF_req_pad_sequences = np.array(sequence.pad_sequences(DF_req_sequences, maxlen=max_seq_len, padding='post', truncating='post'))

del DF_req_sequences

DF_rep_sequences = tok_word.texts_to_sequences(DFreplys)
del DFreplys

for i in range(len(DF_rep_sequences)):
    DF_rep_sequences[i] = [START_TOKEN] + DF_rep_sequences[i] + [END_TOKEN]    

DF_rep_pad_sequences = np.array(sequence.pad_sequences(DF_rep_sequences, maxlen=max_seq_len, padding='post', truncating='post'))

del DF_rep_sequences


In [0]:
n = 1500
#print(DFreplys[n])
print(DF_rep_pad_sequences[n])
print(VOCAB_SIZE)
l = [inv_word_index[i] for i in DF_rep_pad_sequences[n] if i!=0 and i<=len(inv_word_index)]
' '.join(l)

[43124    55   391    33     8  1164     2     6    70  1515    77   161
   197    16     2 43125     0     0     0     0     0     0     0     0
     0     0     0]
43126


"she knows not to steal . i 've taught her better than that ."

In [0]:
########################### decode reply ##########################################

def decode_answer(line, inv_vocab_index):
    l = [inv_word_index[i] for i in line if i!=0  and i<=len(inv_word_index)]
    msg = ' '.join(l)

    return msg


In [0]:
def get_one_hot_seqs(padded_sequences, vocab_size):
    return np.array(list(map(lambda x: to_categorical(x, num_classes=vocab_size), padded_sequences)))

In [0]:
"""X_temp, X_test, y_temp, y_test = train_test_split(DF_req_pad_sequences, DF_rep_pad_sequences, test_size=0.1, random_state=0)

X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, random_state=0)

del DF_req_pad_sequences
del DF_rep_pad_sequences

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

del X_test
del y_test
del X_val
del y_val"""

X_train = DF_req_pad_sequences
y_train = DF_rep_pad_sequences

In [0]:
"""Xoh_train = get_one_hot_seqs(X_train, word_index)
Yoh_train = get_one_hot_seqs(y_train, word_index)

Xoh_val = get_one_hot_seqs(X_val, word_index)
Yoh_val = get_one_hot_seqs(y_val, word_index)"""

'Xoh_train = get_one_hot_seqs(X_train, word_index)\nYoh_train = get_one_hot_seqs(y_train, word_index)\n\nXoh_val = get_one_hot_seqs(X_val, word_index)\nYoh_val = get_one_hot_seqs(y_val, word_index)'

In [0]:
"""p = np.argmax(Yoh_train[1000], axis = -1)
decode_answer(line = p, inv_vocab_index=inv_char_index, dataframe=dataframe)"""

'p = np.argmax(Yoh_train[1000], axis = -1)\ndecode_answer(line = p, inv_vocab_index=inv_char_index, dataframe=dataframe)'

In [0]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')
        

In [0]:
Tx = max_seq_len
Ty = max_seq_len-1

n_a = 150 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 300 # number of units for the post-attention LSTM's hidden state "s"

embed_size = n_s

In [0]:
# Defined shared layers as global variables
repeator1 = RepeatVector(Tx)
concatenator1 = Concatenate(axis=-1)
densor1 = Dense(100, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)
concatenator2 = Concatenate(axis=-1)
repeator2 = RepeatVector(1)
repeator3 = RepeatVector(1)


In [0]:
# GRADED FUNCTION: one_step_attention

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator1(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator1([a, s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    ### END CODE HERE ###
    
    return context

In [0]:
# Please note, this is the post attention LSTM cell.  
# For the purposes of passing the automatic grader
# please do not modify this global variable.  This will be corrected once the automatic grader is also updated.
post_attention_LSTM_cell = LSTM(n_s, return_state = True, name='decoder') # post-attention LSTM 
post_attention_dense = Dense(embed_size) # post-attention LSTM
#post_decoder_dense = Dense(n_s, activation='relu') 
output_layer = Dense(VOCAB_SIZE, activation=softmax)

post_output_dense = Dense(embed_size) # post-attention LSTM

In [0]:
#!wget -O glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
#!sudo unzip glove.840B.300d.zip

In [0]:
"""EMBEDDING_FILE = 'glove.840B.300d.txt'

embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0].lower()
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

n = 0
#prepare embedding matrix
embedding_matrix = np.zeros((VOCAB_SIZE, embed_size))
for word, i in word_index.items():
    #if i >= max_features:
    #    continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        n += 1

print(n, "not found.")

with open("conversations/glove300D_embedding_nltk_tokenizer_"+str(max_word_len)+'words_only_cornell.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(embedding_matrix, f, pickle.HIGHEST_PROTOCOL)
"""

'EMBEDDING_FILE = \'glove.840B.300d.txt\'\n\nembeddings_index = {}\nwith open(EMBEDDING_FILE,encoding=\'utf8\') as f:\n    for line in f:\n        values = line.rstrip().rsplit(\' \')\n        word = values[0].lower()\n        coefs = np.asarray(values[1:], dtype=\'float32\')\n        embeddings_index[word] = coefs\n\nn = 0\n#prepare embedding matrix\nembedding_matrix = np.zeros((VOCAB_SIZE, embed_size))\nfor word, i in word_index.items():\n    #if i >= max_features:\n    #    continue\n    embedding_vector = embeddings_index.get(word)\n    if embedding_vector is not None:\n        # words not found in embedding index will be all-zeros.\n        embedding_matrix[i] = embedding_vector\n    else:\n        n += 1\n\nprint(n, "not found.")\n\nwith open("conversations/glove300D_embedding_nltk_tokenizer_"+str(max_word_len)+\'words_only_cornell.pickle\', \'wb\') as f:\n        # Pickle the \'data\' dictionary using the highest protocol available.\n        pickle.dump(embedding_matrix, f, pick

In [0]:
"""with open("conversations/glove300D_all_embeddings_"+str(max_word_len)+'words_only_cornell.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(embeddings_index, f, pickle.HIGHEST_PROTOCOL)"""

'with open("conversations/glove300D_all_embeddings_"+str(max_word_len)+\'words_only_cornell.pickle\', \'wb\') as f:\n        # Pickle the \'data\' dictionary using the highest protocol available.\n        pickle.dump(embeddings_index, f, pickle.HIGHEST_PROTOCOL)'

In [0]:
with open("conversations/glove300D_embedding_nltk_tokenizer_"+str(max_word_len)+'words_only_cornell.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    embedding_matrix = pickle.load(f)

In [0]:
decoder_layer_norm = LayerNormalization(epsilon=1e-6, name='decoder_layer_norm')
attention_layer_norm = LayerNormalization(epsilon=1e-6, name='attention_layer_norm')
context_layer_norm = LayerNormalization(epsilon=1e-6, name='context_layer_norm')

In [0]:
def one_hot(x):
    x = tf.argmax(x)
    x = tf.one_hot(x, VOCAB_SIZE)
    x = repeator3(x)
    #x = tf.expand_dims(x, axis=1)
    return x

"""
def which(x):
    x = tf.argmax(x, axis=-1)
    x = tf.cast(x, tf.float32)
    #x = tf.reshape(x, (1,1,1))
    x = tf.expand_dims(x, axis=1)
    x = tf.expand_dims(x, axis=1) 
    return x
"""

def which(x):
    x = tf.argmax(x, axis=-1)
    x = tf.expand_dims(x, axis=1) 
    return x

In [0]:
# GRADED FUNCTION: model

def my_model(Tx, Ty, n_a, n_s, vocab_size, embedding_matrix, embed_size=300):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X1 = Input(shape=(Tx, ), name='X1')

    X2 = Input(shape=(Ty, ), name='X2')

    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    #x0 = Input(shape=(1,1,), name='x0')
    x0 = Input(shape=(1,embed_size,), name='x0')

    s = s0
    c = c0
    x = x0

    # Initialize empty list of outputs

    ### START CODE HERE ###
    embLX1 = Embedding(vocab_size, embed_size, mask_zero=True, 
                       weights=[embedding_matrix],
                       embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix), trainable = True)
    embLX2 = Embedding(vocab_size, embed_size, mask_zero=True, 
                       weights=[embedding_matrix],
                      embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix), trainable = True)

    embX1 = embLX1(X1)
    embX2 = embLX2(X2)
    #embX2 = embLX1(X2)
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    enc = Bidirectional(LSTM(units=n_a, return_sequences=True), name='encoder')(embX1)

    enc = LayerNormalization(epsilon=1e-6, name='encoder_layer_norm')(enc)

    outputs = []

    for t in range(Ty):

        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(enc, s)

        #context = post_attention_dense(context) # ??? might revert emb_dim to 300 if current one is not good
        #context = context_layer_norm(context)   # ???
    
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)

        #context = Lambda(lambda context: tf.reshape(context, (-1,-1, embed_size)))(context)
        
        embX2t = Lambda(lambda args: tf.expand_dims(args[0][:,args[1],:], axis=1))((embX2, t))

        #print("e",x)
        
        inputs = concatenator2([context, embX2t, x])
        #inputs = concatenator2([context, embX2t])
        #inputs = context

        inputs = attention_layer_norm(inputs)

        s, _, c = post_attention_LSTM_cell(inputs=inputs, initial_state=[s, c])

        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)

        s = decoder_layer_norm(s)

        out = output_layer(inputs=s)

        #x = Lambda(which2)(out) # might use one_hot on x, then pass through Dense to get shape of embed_size
        x = Lambda(which)(out) # might use one_hot on x, then pass through Dense to get shape of embed_size
        x = embLX2(x)
        # or, could pass ^out directly through Dense to get shape of embed_size     <--------- preferred
        #x = post_output_dense(out)
        #x = Lambda(lambda x: tf.expand_dims(x, axis=1))(x)


        outputs.append(out)

    outputs = Lambda(lambda X: tf.transpose(X, [1, 0, 2]), name='outputs')(outputs)

    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs=[X1,X2,s0,c0, x0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [0]:
VOCAB_SIZE

43126

In [0]:
K.clear_session()

model = my_model(Tx=max_seq_len, Ty=max_seq_len-1 , n_a=n_a, n_s=n_s, 
                 vocab_size=VOCAB_SIZE, embedding_matrix=embedding_matrix, embed_size=embed_size)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X1 (InputLayer)                 [(None, 27)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 27, 300)      12937800    X1[0][0]                         
__________________________________________________________________________________________________
encoder (Bidirectional)         (None, 27, 300)      541200      embedding[0][0]                  
__________________________________________________________________________________________________
s0 (InputLayer)                 [(None, 300)]        0                                            
______________________________________________________________________________________________

In [0]:
"""### START CODE HERE ### (≈2 lines)
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
### END CODE HERE ###"""

'### START CODE HERE ### (≈2 lines)\nopt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)\nmodel.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])\n### END CODE HERE ###'

In [0]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, max_seq_len - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, max_seq_len - 1)
  y_true = tf.reshape(y_true, shape=(-1, max_seq_len - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
learning_rate = CustomSchedule(embed_size)

opt = Adam()#learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


loss_weights = list(np.zeros(Ty))

for i in range(Ty):
    loss_weights[i] = (Ty-i)/Ty
    #loss_weights[i] = 1.0/(i+1)

model.compile(optimizer=opt, loss=loss_function, metrics=[accuracy])#, loss_weights=loss_weights)

#model.summary()

In [0]:
def random_mini_batches(X1, X2, Y, mini_batch_size = 64):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(911-786)            # To make your "random" minibatches the same as ours
    m = X1.shape[0]                  # number of training examples
    mini_batches = []

    n_y = Y.shape[0]
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X1 = X1[permutation]
    shuffled_X2 = X2[permutation]
    shuffled_Y = Y[permutation]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X1 = shuffled_X1[k*mini_batch_size : (k+1) * mini_batch_size]
        mini_batch_X2 = shuffled_X2[k*mini_batch_size : (k+1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[k*mini_batch_size : (k+1) * mini_batch_size]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X1, mini_batch_X2, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X1 = shuffled_X1[num_complete_minibatches*mini_batch_size : ]
        mini_batch_X2 = shuffled_X2[num_complete_minibatches*mini_batch_size : ]
        mini_batch_Y = shuffled_Y[num_complete_minibatches*mini_batch_size : ]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X1, mini_batch_X2, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


In [0]:
X1 = X_train
X2 = y_train[:,:-1]
Y = y_train[:, 1:]
m = X1.shape[0]

s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
#x0 = np.ones((m, 1, 1))*START_TOKEN
x0 = np.zeros((m, 1, embed_size))


In [0]:
"""BATCH_SIZE = 64
BUFFER_SIZE = 200000
# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'X1': X1,
        'X2': X2,
        's0': s0,
        'c0': c0,
        'x0': x0
    },
    {
        'outputs': Y
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)"""

In [0]:
"""#model.load_weights('saved_word_emb3_weights.h5')

for i in range(10):
    model.fit(dataset, #validation_data=validation_data,
                    epochs=1, 
                    shuffle=True, #batch_size=64, 
                    verbose=1)

    model.save('my_word_emb3_model.h5')
    model.save_weights('saved_word_emb3_weights.h5')"""

"#model.load_weights('saved_word_emb3_weights.h5')\n\nfor i in range(10):\n    model.fit(dataset, #validation_data=validation_data,\n                    epochs=1, \n                    shuffle=True, #batch_size=64, \n                    verbose=1)\n\n    model.save('my_word_emb3_model.h5')\n    model.save_weights('saved_word_emb3_weights.h5')"

In [0]:
model.load_weights('saved_word_emb3_weights.h5')

for i in range(10):
    model.fit([X1, X2, s0, c0, x0], Y, #validation_data=validation_data,
                    epochs=1, shuffle=True, batch_size=64, 
                    verbose=1)

    model.save('my_word_emb3_model.h5')
    model.save_weights('saved_word_emb3_weights.h5')

  19/2645 [..............................] - ETA: 33:52 - loss: 4.1108 - accuracy: 0.0799

In [0]:
"""minibatch_size = 1024

#model = load_model('my_word_emb2_model.h5')
#model.load_weights('saved_word_emb2_weights.h5')
y_train = y_train[:,1:]
minibatches = random_mini_batches(X_train, y_train, mini_batch_size = minibatch_size)
del X_train
del y_train

for i in range(100):
    epoch_loss = 0

    num_minibatches = len(minibatches)

    j = 0

    for minibatch in minibatches:
        (minibatch_X, minibatch_Y) = minibatch

        #Yoh_train = get_one_hot_seqs(minibatch_Y, VOCAB_SIZE)

        #outputs = list(Yoh_train.swapaxes(0,1))

        m = minibatch_X.shape[0]
        #valshape = X_val.shape[0]
        #s0 = np.zeros((m, n_s))
        #c0 = np.zeros((m, n_s))
        #x0 = np.ones((m, 1, 1))*START_TOKEN
        #x0 = np.zeros((m, 1, VOCAB_SIZE))

        if j%10 == 0 or True:
            #print("current loss:", epoch_loss)
            history = model.fit([minibatch_X, np.zeros((m, n_s)), np.zeros((m, n_s)), np.ones((m, 1, 1))*START_TOKEN], 
                                list(get_one_hot_seqs(minibatch_Y, VOCAB_SIZE).swapaxes(0,1)), #validation_data=validation_data,
                                #list(minibatch_Y.swapaxes(0,1)),
                                epochs=1,
                                shuffle=True, 
                                batch_size=128,
                                verbose=1
                                )
            model.save('my_word_emb2_model.h5')
            model.save_weights('saved_word_emb2_weights.h5')
        else:
            history = model.fit([minibatch_X, np.zeros((m, n_s)), np.zeros((m, n_s)), np.ones((m, 1, 1))*START_TOKEN], 
                                list(get_one_hot_seqs(minibatch_Y, VOCAB_SIZE).swapaxes(0,1)), #validation_data=validation_data,
                                #list(minibatch_Y.swapaxes(0,1)),          
                                epochs=1,
                                shuffle=True, 
                                batch_size=128,
                                verbose=0
                                )
                            
        epoch_loss += history.history['loss'][-1]

        j += 1

        #del Xoh_train
        #del Yoh_train
        #del outputs
        del minibatch
        del minibatch_X
        del minibatch_Y
        #del s0
        #del c0
        #del x0  

    print("loss after epoch", i, " is", epoch_loss)
    model.save('my_word_emb2_model.h5')
    model.save_weights('saved_word_emb2_weights.h5')  

    del minibatches"""

In [0]:
"""
m = Sequential()
m.add(Embedding(VOCAB_SIZE, embed_size, mask_zero=True, weights=[embedding_matrix],
                       embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix), trainable = True))

input_array = tf.expand_dims([word_index['hello']],0)

#input_array = tf.expand_dims([START_TOKEN]+[END_TOKEN],0)

m.compile('adam', 'accuracy')
out = m.predict(input_array, steps=1)

print(out)

print(embedding_matrix[word_index['hello']])"""